In [5]:
# 最⼩費⽤流(min cost flow)
# プライマルデュアル法 
# sからtへの流量fの最小費用流を求める 流せない場合は-1を返す
# 蟻本p200
def PrimalDual(G,s,t,f):
    V = len(G) # 頂点数
    prev_v = [0 for i in range(V)]
    prev_e = [0 for i in range(V)] # 直前の頂点と辺
    inf = 10**9
    res = 0
    while f > 0:
        #ベルマンフォード法で最短路を求める(負の経路が存在するため)
        dist=[inf]*V; #最短距離
        dist[s] = 0
        update = True
        while update:
            update = False
            for v in range(V):
                if dist[v] == inf: continue
                for i in range(len(G[v])):
                    if G[v][i][1] > 0 and dist[G[v][i][0]] > dist[v] + G[v][i][2]: #最短距離を更新
                        dist[G[v][i][0]] = dist[v] + G[v][i][2]
                        prev_v[G[v][i][0]] = v #直前の頂点、辺も記録
                        prev_e[G[v][i][0]] = i
                        update = True

        if dist[t] == inf: #これ以上流せないなら-1
            return -1

        d = f #流量
        v = t
        #最短路に沿ってできる限り流すというのを繰り返す
        while  v != s:
            d = min(d, G[prev_v[v]][prev_e[v]][1])
            v = prev_v[v]
        
        f -= d
        res += d*dist[t]
        v = t
        while  v != s:
            G[prev_v[v]][prev_e[v]][1] -= d
            G[v][G[prev_v[v]][prev_e[v]][3]][1] += d
            v = prev_v[v]
            
    return res


# アルゴリズム2021 
# 12最小費用流の例

V=4 # 頂点数
G=[[] for i in range(V)] # グラフの隣接リスト表現
# fからtへ向かう容量cap、コストcostの辺をグラフに追加する
def add_edge(f,t,cap,cost):
    G[f].append([t,cap,cost,len(G[t])]) # 辺を表す (行き先、容量、コスト、逆辺)
    G[t].append([f,0,-cost,len(G[f])-1])

add_edge(0, 1, 3, 6)
add_edge(0, 2, 5, 2)
add_edge(1, 3, 4, 3)
add_edge(2, 1, 7, 3)
add_edge(2, 3, 6, 9)

PrimalDual(G,0,3,7)

67

In [6]:
#Algo 11-2
#スタートから各文字列に流量a、コストiのエッジ、各文字列から各アルファベットに流量(各文字列中の各アルファベットの数)、コスト0のエッジ、
#各アルファベットからゴールに流量(各アルファベットの必要数)、コスト0のエッジを引き、プライマルデュアル法により最小費用流を求める
lines=[' ' for i in range(5)]
lines[0]='aabcd'
lines[1]='3'
lines[2]='aca 2'
lines[3]='bda 3'
lines[4]='ab 10'

T=lines[0]
N=int(lines[1])
INF=10**9
V=1+N+26+1 #頂点数
G=[[] for i in range(V)] #グラフ
prev_v=[0 for i in range(V)] #直前の頂点
prev_e=[0 for i in range(V)] #直前の辺
for i in range(N):
    s,a=[x.strip() for x in lines[i+2].split()]
    add_edge(0,i+1, int(a), i+1) #スタートから各文字列に流量a、コストiのエッジを引く
    for j in range(26):
        if s.count(chr(ord('a')+j))!=0:
            add_edge(i+1,N+1+j, s.count(chr(ord('a')+j)), 0) #各文字列から各アルファベットに流量(各文字列中の各アルファベットの数)、コスト0のエッジを引く

for i in range(26):
    if T.count(chr(ord('a')+i))!=0:
        add_edge(N+1+i,V-1, T.count(chr(ord('a')+i)), 0) #各アルファベットからゴールに流量(各アルファベットの必要数)、コスト0のエッジを引く

#print(G)
print(PrimalDual(G,0,V-1,len(T)))

8
